## Approche DL pour Parkinson
---

In [ ]:
# importer les bibliotheques necessaires
import os # interaction systeme
import librosa # bibliotheque audio, spectro et extraction feature
import numpy as np # tableau 
import pandas as pd # data frame
import matplotlib.pyplot as plt # visualisation

In [ ]:
# definir les chemins des dossiers contenant les fichiers audio
path_to_hc = 'HC_AH/'
path_to_pd = 'PD_AH/'

In [ ]:
# Charger les fichiers audio yo pour les patients sains
y_hc = []
sr_hc = []

for file in os.listdir(path_to_hc):
    if file.endswith('.wav'):
        audio_path = os.path.join(path_to_hc, file)
        y, sr = librosa.load(audio_path, sr=None)
        y_hc.append(y)
        sr_hc.append(sr)

print('Nombre de patients sains:', len(y_hc))

# Charger les fichiers audio pour les patients parkinsoniens
y_pd = []
sr_pd = []

for file in os.listdir(path_to_pd):
    if file.endswith('.wav'):
        audio_path = os.path.join(path_to_pd, file)
        y, sr = librosa.load(audio_path, sr=None)
        y_pd.append(y)
        sr_pd.append(sr)

print('Nombre de patients parkinsoniens:', len(y_pd))

In [ ]:
# On fixe la durée car pas la même durée pour tous les fichiers audio
fixed_duration = 1.5

# convertir un signal audio en spectrogramme de taille fixe
def audio_to_fixed_spectrogram(y, sr, n_mels=128, fixed_length=None):
    if fixed_length is not None:
        y = librosa.util.fix_length(y, size=fixed_length) # Redimensionner le signal audio à une durée fixe
    # Calculer le spectrogramme Mel
    S = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=n_mels)
    # Convertir en échelle logarithmique
    S_db = librosa.power_to_db(S, ref=np.max)
    return S_db

In [ ]:
# longueur fixe en échantillons
fixed_length = int(fixed_duration * sr_hc[1])
print(fixed_length)

In [ ]:
# Convertir tous les fichiers audio en spectrogrammes de la taille fixe définie
X_hc = []
for y, sr in zip(y_hc, sr_hc):
    spectrogram = audio_to_fixed_spectrogram(y, sr, fixed_length=fixed_length)
    X_hc.append(spectrogram)

X_pd = []
for y, sr in zip(y_pd, sr_pd):
    spectrogram = audio_to_fixed_spectrogram(y, sr, fixed_length=fixed_length)
    X_pd.append(spectrogram)

In [ ]:
# Convertir en tableaux NumPy
X_hc = np.array(X_hc)
X_pd = np.array(X_pd)

In [ ]:
# Verif les dimensions
print("Shape de X_hc :", X_hc.shape)
print("Shape de X_pd :", X_pd.shape)

# Afficher un exemple de spectrogramme
plt.figure(figsize=(10, 4))
plt.imshow(X_hc[14], aspect='auto', origin='lower')
plt.colorbar(format='%+2.0f dB')
plt.title('Spectrogramme Mel pour un patient sain')
plt.ylabel('Echelle de Mel')
plt.xlabel('Trame temporelle')
plt.tight_layout()
plt.show()

In [ ]:
# Afficher un exemple de spectrogramme
plt.figure(figsize=(10, 4))
plt.imshow(X_pd[14], aspect='auto', origin='lower')
plt.colorbar(format='%+2.0f dB')
plt.title('Spectrogramme Mel pour un patient parkinsonien')
plt.ylabel('Echelle de Mel')
plt.xlabel('Trame temporelle')
plt.tight_layout()
plt.show()

In [ ]:
# Ajouter une dimension pour les canaux pour pouvoir utiliser CNN
X_hc = X_hc[..., np.newaxis]
X_pd = X_pd[..., np.newaxis]

# Verif les dimensions
print("Shape de X_hc après ajout de canal :", X_hc.shape)
print("Shape de X_pd après ajout de canal :", X_pd.shape)

In [ ]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

# definir les labels de classes
y_hc_labels = np.zeros(len(X_hc))  # 0 pour les patients sains
y_pd_labels = np.ones(len(X_pd))   # 1 pour les patients parkinsoniens

In [ ]:
# Combiner les données
X = np.concatenate((X_hc, X_pd), axis=0)
y = np.concatenate((y_hc_labels, y_pd_labels), axis=0)

# Convertir les labels en catégories (one-hot encoding)
y = to_categorical(y, num_classes=2)

In [ ]:
# Diviser les données en training set et test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

print("Shape de X_train :", X_train.shape)
print("Shape de X_test :", X_test.shape)
print("Shape de y_train :", y_train.shape)
print("Shape de y_test :", y_test.shape)

---

Modèle CNN classique

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Architecture du CNN
model = Sequential([
    # Couche de convolution 2D
    Conv2D(32, (3, 3), activation='relu', input_shape=X_train.shape[1:]),
    # Couche de pooling 2D pour réduire la dimensionnalité en gardant les informations importantes
    MaxPooling2D((2, 2)),
    
    # Deuxième couche de convolution et pooling
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    
    # Troisième couche de convolution et pooling
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    
    # Flatten pour convertir les matrices 2D des couches précédentes en vecteur 1D pour la couche fully connected
    Flatten(),
    
    # Couche fully connected avec 128 neurones et fonction d'activation ReLU pour apprendre des fonctions non linéaires
    Dense(128, activation='relu'),
    Dropout(0.5),  # Dropout pour éviter le surapprentissage
    
    # Couche dense avec 2 neurones et fonction d'activation softmax pour la classification des 2 classes
    Dense(2, activation='softmax')  # 2 classes (sain, parkinsonien)
])


In [ ]:
# Compiler le modèle
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Afficher un résumé du modèle
model.summary()

In [ ]:
# Entraîner le modèle
history = model.fit(
    X_train, y_train,
    epochs=70,  # Nombre d'époques
    batch_size=32,  # Taille du batch
    validation_data=(X_test, y_test)
)

In [ ]:
# Visualiser les courbes d'apprentissage

plt.plot(history.history['accuracy'], label='Accuracy (entraînement)')
plt.plot(history.history['val_accuracy'], label='Accuracy (validation)')
plt.xlabel('Époques')
plt.ylabel('Accuracy')
plt.title("Courbes d'apprentissage du CNN classique")
plt.legend()
plt.show()

In [ ]:
# Évaluer le modèle sur les données de test
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print("Accuracy sur le test set :", test_accuracy)

# Faire des prédictions sur le test set
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(y_test, axis=1)

In [ ]:
# Matrice de confusion
from sklearn.metrics import confusion_matrix
import seaborn as sns

conf_matrix = confusion_matrix(y_true_classes, y_pred_classes)
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=['Sain', 'Parkinsonien'], yticklabels=['Sain', 'Parkinsonien'])
plt.xlabel('Prédit')
plt.ylabel('Réel')
plt.title('Matrice de confusion du CNN classique')
plt.show()

---

Modèle CNN avec des couches plus profondes

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Architecture du CNN profond
model1 = Sequential([
    # Première couche de convolution 2D
    Conv2D(64, (3, 3), activation='relu', input_shape=X_train.shape[1:]),
    # Couche de pooling 2D pour réduire la dimensionnalité
    MaxPooling2D((2, 2)),
    
    # Deuxième couche de convolution 2D
    Conv2D(128, (3, 3), activation='relu'),
    # Couche de pooling 2D
    MaxPooling2D((2, 2)),
    
    # Troisième couche de convolution 2D
    Conv2D(256, (3, 3), activation='relu'),
    # Couche de pooling 2D
    MaxPooling2D((2, 2)),
    
    # Aplatir les données pour les couches fully connected
    Flatten(),
    
    # Couche fully connected avec 256 neurones
    Dense(256, activation='relu'),
    # Dropout pour éviter le surapprentissage
    Dropout(0.5),
    
    # Couche de sortie avec 2 neurones (classification binaire)
    Dense(2, activation='softmax')
])

# Compiler le modèle
model1.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Afficher un résumé du modèle
model1.summary()

# Entraîner le modèle
history1 = model1.fit(
    X_train, y_train,
    epochs=60,  # Nombre d'époques
    batch_size=32,  # Taille du batch
    validation_data=(X_test, y_test)
)

# Évaluer le modèle sur les données de test
test_loss, test_accuracy = model1.evaluate(X_test, y_test)
print("Accuracy sur le test set :", test_accuracy)

# Faire des prédictions sur le test set
y_pred = model1.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(y_test, axis=1)

import matplotlib.pyplot as plt

# Tracer la précision 
plt.plot(history1.history['accuracy'], label='Accuracy (entraînement)')
plt.plot(history1.history['val_accuracy'], label='Accuracy (validation)')
plt.xlabel('Époques')
plt.ylabel('Accuracy')
plt.title("Courbes d'apprentissage du CNN profond")
plt.legend()
plt.show()

# Tracer la perte
plt.plot(history1.history['loss'], label='Loss (entraînement)')
plt.plot(history1.history['val_loss'], label='Loss (validation)')
plt.xlabel('Époques')
plt.ylabel('Loss')
plt.title("Perte du CNN profond")
plt.legend()
plt.show()

---

Modèle CNN avec une régularisation L2

In [ ]:
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.regularizers import l2

# Architecture du CNN avec Batch Normalization
model2 = Sequential([
    # Première couche de convolution 2D
    Conv2D(32, (3, 3), activation='relu', kernel_regularizer=l2(0.001), input_shape=X_train.shape[1:]),
    BatchNormalization(), # Normalisation des activations
    MaxPooling2D((2, 2)), # Couche de pooling 2D
    
    # Deuxième couche de convolution 2D
    Conv2D(64, (3, 3), activation='relu', kernel_regularizer=l2(0.001)),
    BatchNormalization(), # Normalisation des activations
    MaxPooling2D((2, 2)), # Couche de pooling 2D
    
    # Troisième couche de convolution 2D
    Conv2D(128, (3, 3), activation='relu', kernel_regularizer=l2(0.001)),
    BatchNormalization(), # Normalisation des activations
    MaxPooling2D((2, 2)), # Couche de pooling 2D
    
    # Aplatir les données pour les couches fully connected
    Flatten(),
    
    # Couche fully connected avec 128 neurones
    Dense(128, activation='relu', kernel_regularizer=l2(0.001)),
    BatchNormalization(), # Normalisation des activations
    Dropout(0.5), # Dropout pour éviter le surapprentissage

    # Couche de sortie avec 2 neurones (classification binaire)
    Dense(2, activation='softmax')
])

# Compiler le modèle
model2.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Afficher un résumé du modèle
model2.summary()

# Entraîner le modèle
history2 = model2.fit(
    X_train, y_train,
    epochs=100,
    batch_size=32,
    validation_data=(X_test, y_test)
)

# Tracer la précision
plt.plot(history2.history['accuracy'], label='Accuracy (entraînement)')
plt.plot(history2.history['val_accuracy'], label='Accuracy (validation)')
plt.xlabel('Époques')
plt.ylabel('Accuracy')
plt.title("Courbes d'apprentissage du CNN avec régularisation L2")
plt.legend()
plt.show()

# Tracer la perte 
plt.plot(history2.history['loss'], label='Loss (entraînement)')
plt.plot(history2.history['val_loss'], label='Loss (validation)')
plt.xlabel('Époques')
plt.ylabel('Loss')
plt.title("Perte du CNN profond avec Batch Normalization et régularisation L2")
plt.legend()
plt.show()

---

Modèle CNN avec des couches séparables

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SeparableConv2D, MaxPooling2D, Flatten, Dense, Dropout

# Architecture du CNN avec couches séparables
model4 = Sequential([
    # Première couche de convolution séparable
    SeparableConv2D(32, (3, 3), activation='relu', input_shape=X_train.shape[1:]),
    # Couche de pooling 2D pour réduire la dimensionnalité
    MaxPooling2D((2, 2)),
    
    # Deuxième couche de convolution séparable
    SeparableConv2D(64, (3, 3), activation='relu'),
    # Couche de pooling 2D
    MaxPooling2D((2, 2)),
    
    # Troisième couche de convolution séparable
    SeparableConv2D(128, (3, 3), activation='relu'),
    # Couche de pooling 2D
    MaxPooling2D((2, 2)),
    
    # Aplatir les données pour les couches fully connected
    Flatten(),
    
    # Couche fully connected avec 128 neurones
    Dense(128, activation='relu'),
    # Dropout pour éviter le surapprentissage
    Dropout(0.5),
    
    # Couche de sortie avec 2 neurones (classification binaire)
    Dense(2, activation='softmax')
])

# Compiler le modèle
model4.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Afficher un résumé du modèle
model4.summary()

# Entraîner le modèle
history4 = model4.fit(
    X_train, y_train,
    epochs=70,  # Nombre d'époques
    batch_size=32,  # Taille du batch
    validation_data=(X_test, y_test)
)

# Évaluer le modèle sur les données de test
test_loss, test_accuracy = model4.evaluate(X_test, y_test)
print("Accuracy sur le test set :", test_accuracy)

# Faire des prédictions sur le test set
y_pred = model4.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(y_test, axis=1)

import matplotlib.pyplot as plt

# Tracer la précision
plt.plot(history4.history['accuracy'], label='Accuracy (entraînement)')
plt.plot(history4.history['val_accuracy'], label='Accuracy (validation)')
plt.xlabel('Époques')
plt.ylabel('Accuracy')
plt.title("Courbes d'apprentissage du CNN avec couches séparables")
plt.legend()
plt.show()

# Tracer la perte
plt.plot(history4.history['loss'], label='Loss (entraînement)')
plt.plot(history4.history['val_loss'], label='Loss (validation)')
plt.xlabel('Époques')
plt.ylabel('Loss')
plt.title("Perte du CNN avec couches séparables")
plt.legend()
plt.show()

---

Modèle CNN allégé en paramètres

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SeparableConv2D, MaxPooling2D, GlobalAveragePooling2D, Flatten, Dense, Dropout

# Architecture du CNN léger avec couches séparables
model_light = Sequential([
    # Première couche de convolution séparable
    SeparableConv2D(32, (3, 3), activation='relu', input_shape=X_train.shape[1:]),
    # Couche de pooling 2D pour réduire la dimensionnalité
    MaxPooling2D((2, 2)),
    
    # Deuxième couche de convolution séparable
    SeparableConv2D(64, (3, 3), activation='relu'),
    # Couche de pooling 2D
    MaxPooling2D((2, 2)),
    
    # Troisième couche de convolution séparable
    SeparableConv2D(128, (3, 3), activation='relu'),
    # Global Average Pooling pour éviter la réduction excessive
    GlobalAveragePooling2D(),
    
    # Couche fully connected avec 128 neurones
    Dense(128, activation='relu'),
    # Dropout pour éviter le surapprentissage
    Dropout(0.5),
    
    # Couche de sortie avec 2 neurones (classification binaire)
    Dense(2, activation='softmax')
])

# Compiler le modèle
model_light.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Afficher un résumé du modèle
model_light.summary()

# Entraîner le modèle
history_light = model_light.fit(
    X_train, y_train,
    epochs=100,  # Nombre d'époques
    batch_size=32,  # Taille du batch
    validation_data=(X_test, y_test)
)

# Évaluer le modèle sur les données de test
test_loss, test_accuracy = model_light.evaluate(X_test, y_test)
print("Accuracy sur le test set :", test_accuracy)

# Faire des prédictions sur le test set
y_pred = model_light.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(y_test, axis=1)

import matplotlib.pyplot as plt

# Tracer la précision
plt.plot(history_light.history['accuracy'], label='Accuracy (entraînement)')
plt.plot(history_light.history['val_accuracy'], label='Accuracy (validation)')
plt.xlabel('Époques')
plt.ylabel('Accuracy')
plt.title("Courbes d'apprentissage du CNN léger")
plt.legend()
plt.show()

# Tracer la perte
plt.plot(history_light.history['loss'], label='Loss (entraînement)')
plt.plot(history_light.history['val_loss'], label='Loss (validation)')
plt.xlabel('Époques')
plt.ylabel('Loss')
plt.title("Perte du CNN léger")
plt.legend()
plt.show()

---

VGG16

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SeparableConv2D, MaxPooling2D, GlobalAveragePooling2D, Flatten, Dense, Dropout

# Architecture du CNN inspirée de VGG16 avec couches séparables
model_vgg16 = Sequential([
    # Bloc 1
    SeparableConv2D(64, (3, 3), activation='relu', padding='same', input_shape=X_train.shape[1:]),
    SeparableConv2D(64, (3, 3), activation='relu', padding='same'),
    MaxPooling2D((2, 2)),
    
    # Bloc 2
    SeparableConv2D(128, (3, 3), activation='relu', padding='same'),
    SeparableConv2D(128, (3, 3), activation='relu', padding='same'),
    MaxPooling2D((2, 2)),
    
    # Bloc 3
    SeparableConv2D(256, (3, 3), activation='relu', padding='same'),
    SeparableConv2D(256, (3, 3), activation='relu', padding='same'),
    SeparableConv2D(256, (3, 3), activation='relu', padding='same'),
    MaxPooling2D((2, 2)),
    
    # Bloc 4
    SeparableConv2D(512, (3, 3), activation='relu', padding='same'),
    SeparableConv2D(512, (3, 3), activation='relu', padding='same'),
    SeparableConv2D(512, (3, 3), activation='relu', padding='same'),
    GlobalAveragePooling2D(),  # Remplace MaxPooling2D pour éviter la réduction excessive
    
    # Couche fully connected
    Dense(4096, activation='relu'),
    Dropout(0.5),  # Dropout pour éviter le surapprentissage
    
    # Couche de sortie avec 2 neurones (classification binaire)
    Dense(2, activation='softmax')
])

# Compiler le modèle
model_vgg16.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Afficher un résumé du modèle
model_vgg16.summary()

# Entraîner le modèle
history_vgg16 = model_vgg16.fit(
    X_train, y_train,
    epochs=100,  # Nombre d'époques
    batch_size=32,  # Taille du batch
    validation_data=(X_test, y_test)
)

# Évaluer le modèle sur les données de test
test_loss, test_accuracy = model_vgg16.evaluate(X_test, y_test)
print("Accuracy sur le test set :", test_accuracy)

# Faire des prédictions sur le test set
y_pred = model_vgg16.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(y_test, axis=1)

In [ ]:
import matplotlib.pyplot as plt

# Tracer la précision
plt.plot(history_vgg16.history['accuracy'], label='Accuracy (entraînement)')
plt.plot(history_vgg16.history['val_accuracy'], label='Accuracy (validation)')
plt.xlabel('Époques')
plt.ylabel('Accuracy')
plt.title("Courbes d'apprentissage du CNN inspiré de VGG16")
plt.legend()
plt.show()

# Tracer la perte
plt.plot(history_vgg16.history['loss'], label='Loss (entraînement)')
plt.plot(history_vgg16.history['val_loss'], label='Loss (validation)')
plt.xlabel('Époques')
plt.ylabel('Loss')
plt.title("Perte du CNN inspiré de VGG16")
plt.legend()
plt.show()

---

VGG19

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SeparableConv2D, MaxPooling2D, GlobalAveragePooling2D, Flatten, Dense, Dropout

# Architecture du CNN inspirée de VGG19 avec couches séparables
model_vgg19 = Sequential([
    # Bloc 1
    SeparableConv2D(64, (3, 3), activation='relu', padding='same', input_shape=X_train.shape[1:]),
    SeparableConv2D(64, (3, 3), activation='relu', padding='same'),
    MaxPooling2D((2, 2)),
    
    # Bloc 2
    SeparableConv2D(128, (3, 3), activation='relu', padding='same'),
    SeparableConv2D(128, (3, 3), activation='relu', padding='same'),
    MaxPooling2D((2, 2)),
    
    # Bloc 3
    SeparableConv2D(256, (3, 3), activation='relu', padding='same'),
    SeparableConv2D(256, (3, 3), activation='relu', padding='same'),
    SeparableConv2D(256, (3, 3), activation='relu', padding='same'),
    SeparableConv2D(256, (3, 3), activation='relu', padding='same'),
    MaxPooling2D((2, 2)),
    
    # Bloc 4
    SeparableConv2D(512, (3, 3), activation='relu', padding='same'),
    SeparableConv2D(512, (3, 3), activation='relu', padding='same'),
    SeparableConv2D(512, (3, 3), activation='relu', padding='same'),
    SeparableConv2D(512, (3, 3), activation='relu', padding='same'),
    GlobalAveragePooling2D(),  # Remplace MaxPooling2D pour éviter la réduction excessive
    
    # Couche fully connected
    Dense(4096, activation='relu'),
    Dropout(0.5),  # Dropout pour éviter le surapprentissage
    
    # Couche de sortie avec 2 neurones (classification binaire)
    Dense(2, activation='softmax')
])

# Compiler le modèle
model_vgg19.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Afficher un résumé du modèle
model_vgg19.summary()

# Entraîner le modèle
history_vgg19 = model_vgg19.fit(
    X_train, y_train,
    epochs=100,  # Nombre d'époques
    batch_size=32,  # Taille du batch
    validation_data=(X_test, y_test)
)

# Évaluer le modèle sur les données de test
test_loss, test_accuracy = model_vgg19.evaluate(X_test, y_test)
print("Accuracy sur le test set :", test_accuracy)

# Faire des prédictions sur le test set
y_pred = model_vgg19.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(y_test, axis=1)

import matplotlib.pyplot as plt

# Tracer la précision
plt.plot(history_vgg19.history['accuracy'], label='Accuracy (entraînement)')
plt.plot(history_vgg19.history['val_accuracy'], label='Accuracy (validation)')
plt.xlabel('Époques')
plt.ylabel('Accuracy')
plt.title("Courbes d'apprentissage du CNN inspiré de VGG19")
plt.legend()
plt.show()

# Tracer la perte
plt.plot(history_vgg19.history['loss'], label='Loss (entraînement)')
plt.plot(history_vgg19.history['val_loss'], label='Loss (validation)')
plt.xlabel('Époques')
plt.ylabel('Loss')
plt.title("Perte du CNN inspiré de VGG19")
plt.legend()
plt.show()

---

Modèle inspiré de ResNet

In [ ]:
from tensorflow.keras.layers import Input, Add, Activation, BatchNormalization, SeparableConv2D, MaxPooling2D, GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model

# Fonction pour créer un bloc résiduel
def residual_block(x, filters, kernel_size=(3, 3), strides=(1, 1)):
    # Branche principale
    # Convolution 2D séparable
    y = SeparableConv2D(filters, kernel_size, strides=strides, padding='same')(x)
    y = BatchNormalization()(y) # Normalisation des activations
    y = Activation('relu')(y) # Fonction d'activation ReLU
    
    y = SeparableConv2D(filters, kernel_size, padding='same')(y) # Convolution 2D séparable
    y = BatchNormalization()(y) # Normalisation des activations
    
    # Connexion skip
    if strides != (1, 1) or x.shape[-1] != filters:
        x = SeparableConv2D(filters, (1, 1), strides=strides, padding='same')(x) # Convolution 2D séparable
        x = BatchNormalization()(x) # Normalisation des activations
    
    # Ajouter la branche principale et la connexion skip
    out = Add()([x, y]) # Ajout des deux branches
    out = Activation('relu')(out) # Fonction d'activation ReLU
    return out

# Modèle ResNet-like
inputs = Input(shape=X_train.shape[1:])  # Entrée du modèle
x = SeparableConv2D(64, (7, 7), strides=(2, 2), padding='same')(inputs)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)

# Blocs résiduels
x = residual_block(x, filters=64)
x = residual_block(x, filters=64)
x = residual_block(x, filters=128, strides=(2, 2))
x = residual_block(x, filters=128)
x = residual_block(x, filters=256, strides=(2, 2))
x = residual_block(x, filters=256)

# Global Average Pooling pour réduire la dimensionnalité
x = GlobalAveragePooling2D()(x)

# Couche fully connected
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)  # Dropout pour éviter le surapprentissage

# Couche de sortie
outputs = Dense(2, activation='softmax')(x)  # Classification binaire

# Créer le modèle
model_resnet = Model(inputs, outputs)

# Compiler le modèle
model_resnet.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Afficher un résumé du modèle
model_resnet.summary()

# Entraîner le modèle
history_resnet = model_resnet.fit(
    X_train, y_train,
    epochs=100,  # Nombre d'époques
    batch_size=32,  # Taille du batch
    validation_data=(X_test, y_test)
)

# Évaluer le modèle sur les données de test
test_loss, test_accuracy = model_resnet.evaluate(X_test, y_test)
print("Accuracy sur le test set :", test_accuracy)

# Faire des prédictions sur le test set
y_pred = model_resnet.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(y_test, axis=1)

import matplotlib.pyplot as plt

# Tracer la précision
plt.plot(history_resnet.history['accuracy'], label='Accuracy (entraînement)')
plt.plot(history_resnet.history['val_accuracy'], label='Accuracy (validation)')
plt.xlabel('Époques')
plt.ylabel('Accuracy')
plt.title("Courbes d'apprentissage du CNN ResNet-like")
plt.legend()
plt.show()

# Tracer la perte
plt.plot(history_resnet.history['loss'], label='Loss (entraînement)')
plt.plot(history_resnet.history['val_loss'], label='Loss (validation)')
plt.xlabel('Époques')
plt.ylabel('Loss')
plt.title("Perte du CNN ResNet-like")
plt.legend()
plt.show()

---

InceptionV3

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt

In [ ]:
# Dupliquer le canal unique pour créer 3 canaux
X_train_rgb = np.repeat(X_train, 3, axis=-1)
X_test_rgb = np.repeat(X_test, 3, axis=-1)

# Vérifier la nouvelle forme
print("Shape de X_train_rgb :", X_train_rgb.shape)
print("Shape de X_test_rgb :", X_test_rgb.shape)

# Redimensionner les spectrogrammes à 128x128 pixels
X_train_resized = tf.image.resize(X_train_rgb, [128, 128])
X_test_resized = tf.image.resize(X_test_rgb, [128, 128])

# Vérifier la nouvelle forme
print("Shape de X_train_resized :", X_train_resized.shape)
print("Shape de X_test_resized :", X_test_resized.shape)

In [ ]:
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Model

# Charger InceptionV3 avec la nouvelle taille d'entrée
model5 = InceptionV3(
    include_top=False,  # Ne pas inclure la couche fully connected finale
    input_shape=(128, 128, 3),  # Nouvelle forme avec 3 canaux
    pooling='avg',  # Global Average Pooling pour réduire la dimensionnalité
    weights='imagenet'  # Utiliser les poids préentraînés sur ImageNet
)

# Geler les couches du modèle préentraîné
model5.trainable = False

# Ajouter une couche fully connected pour la classification
x = model5.output
x = Dense(128, activation='relu')(x)  # Couche fully connected
x = Dropout(0.5)(x)  # Dropout pour éviter le surapprentissage
predictions = Dense(2, activation='softmax')(x)  # Couche de sortie

# Créer le modèle final
model5_final = Model(inputs=model5.input, outputs=predictions)

# Compiler le modèle
model5_final.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Afficher un résumé du modèle
model5_final.summary()

# Entraîner le modèle
history5 = model5_final.fit(
    X_train_resized, y_train,
    epochs=100,
    batch_size=32,
    validation_data=(X_test_resized, y_test)
)


# Visualiser les courbes d'apprentissage
plt.plot(history5.history['accuracy'], label='Accuracy (entraînement)')
plt.plot(history5.history['val_accuracy'], label='Accuracy (validation)')
plt.xlabel('Époques')
plt.ylabel('Accuracy')
plt.title("Courbes d'apprentissage d'InceptionV3")
plt.legend()
plt.show()

In [ ]:
# Évaluer le modèle sur les données de test
test_loss, test_accuracy = model5_final.evaluate(X_test_resized, y_test)
print("Accuracy sur le test set :", test_accuracy)

# Faire des prédictions sur le test set
y_pred = model5_final.predict(X_test_resized)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(y_test, axis=1)

---

InceptionV3 avec une taille de spectrogramme 600x600pixels

In [ ]:
# Dupliquer le canal unique pour créer 3 canaux
X_train_rgb = np.repeat(X_train, 3, axis=-1)
X_test_rgb = np.repeat(X_test, 3, axis=-1)

# Vérifier la nouvelle forme
print("Shape de X_train_rgb :", X_train_rgb.shape)
print("Shape de X_test_rgb :", X_test_rgb.shape)

# Redimensionner les spectrogrammes à 128x128 pixels
X_train_resized = tf.image.resize(X_train_rgb, [600, 600])
X_test_resized = tf.image.resize(X_test_rgb, [600, 600])

# Vérifier la nouvelle forme
print("Shape de X_train_resized :", X_train_resized.shape)
print("Shape de X_test_resized :", X_test_resized.shape)

In [ ]:
# Charger InceptionV3 avec la nouvelle taille d'entrée
base_model = InceptionV3(
    include_top=False,  # Ne pas inclure la couche fully connected finale
    input_shape=(600, 600, 3),  # Nouvelle forme avec 3 canaux
    weights='imagenet'  # Utiliser les poids préentraînés sur ImageNet
)

# Geler les couches du modèle préentraîné
base_model.trainable = False

# Ajouter une couche fully connected pour la classification
x = base_model.output
x = GlobalAveragePooling2D()(x)  # Aplatir les dimensions spatiales
x = Dense(128, activation='relu')(x)  # Couche fully connected
x = Dropout(0.5)(x)  # Dropout pour éviter le surapprentissage
predictions = Dense(2, activation='softmax')(x)  # Couche de sortie

# Créer le modèle final
model5_final = Model(inputs=base_model.input, outputs=predictions)

# Compiler le modèle
model5_final.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Afficher un résumé du modèle
model5_final.summary()

# Entraîner le modèle
history5 = model5_final.fit(
    X_train_resized, y_train,
    epochs=10,  # Nombre d'époques
    batch_size=4,  # Taille du batch
    validation_data=(X_test_resized, y_test)
)

# Visualiser les courbes d'apprentissage
plt.plot(history5.history['accuracy'], label='Accuracy (entraînement)')
plt.plot(history5.history['val_accuracy'], label='Accuracy (validation)')
plt.xlabel('Époques')
plt.ylabel('Accuracy')
plt.title("Courbes d'apprentissage d'InceptionV3")
plt.legend()
plt.show()

In [ ]:
# Évaluer le modèle sur les données de test
test_loss, test_accuracy = model5_final.evaluate(X_test_resized, y_test)
print("Accuracy sur le test set :", test_accuracy)

# Faire des prédictions sur le test set
y_pred = model5_final.predict(X_test_resized)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(y_test, axis=1)

---